### Real-time Face Mask Detection with Python

Indoor places, such as restaurants and grocery stores, are legally required to have rules in place for the mandatory use of face masks. Having a worker manually examining each person to make sure their mask is on simply defeats the goal of limiting contact with people as much as possible. So, a real-time face mask detection system can be used to address this issue that will not only maximize efficiency but will also ensure to potentially save lives.

The technology behind the real-time face mask detection system is not new. In Machine Learning, face mask detection is the problem of computer vision. Too often we see computer vision applications of this technology in our daily lives. A common example is a face unlocking in smartphones.

The goal of a face mask detection system is to create an image recognition system that understands how image classification works, and it should work with great accuracy so that our model can be applied in the realtime situations. It will work by recognizing the boundaries of the face and predicting whether or not you are wearing a face mask in real-time.

#### Convolutional Neural Network (CNN)
Let's create a convolutional neural network to create a real-time facial mask detection model with Python. Here, we will use three dense layers in our model with respectively 50, 35 and finally 2 neurons. The dense network produces the probability of the binary classification of no mask = 1 and mask = 0

In [2]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [3]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [4]:
TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [5]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [6]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

C:\Users\hp\AppData\Local\Temp\ipykernel_22076\3620006576.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
132/132 [==============================] - ETA: 0s - loss: 0.6689 - acc: 0.6015

INFO:tensorflow:Assets written to: model2-001.model\assets


INFO:tensorflow:Assets written to: model2-001.model\assets


132/132 [==============================] - 152s 1s/step - loss: 0.6689 - acc: 0.6015 - val_loss: 0.2568 - val_acc: 0.9072
Epoch 2/10
132/132 [==============================] - ETA: 0s - loss: 0.3700 - acc: 0.8616

...

INFO:tensorflow:Assets written to: model2-005.model\assets


132/132 [==============================] - 168s 1s/step - loss: 0.2457 - acc: 0.9148 - val_loss: 0.0903 - val_acc: 0.9639
Epoch 6/10
132/132 [==============================] - 161s 1s/step - loss: 0.2046 - acc: 0.9240 - val_loss: 0.1566 - val_acc: 0.9175
Epoch 7/10
132/132 [==============================] - ETA: 0s - loss: 0.2235 - acc: 0.9270

INFO:tensorflow:Assets written to: model2-007.model\assets


INFO:tensorflow:Assets written to: model2-007.model\assets


132/132 [==============================] - 165s 1s/step - loss: 0.2235 - acc: 0.9270 - val_loss: 0.0375 - val_acc: 0.9845
Epoch 8/10
132/132 [==============================] - 156s 1s/step - loss: 0.1915 - acc: 0.9293 - val_loss: 0.0389 - val_acc: 0.9948
Epoch 9/10
132/132 [==============================] - 159s 1s/step - loss: 0.1980 - acc: 0.9293 - val_loss: 0.0629 - val_acc: 0.9742
Epoch 10/10
132/132 [==============================] - 157s 1s/step - loss: 0.1723 - acc: 0.9384 - val_loss: 0.0409 - val_acc: 0.9845


In [10]:
model.save('model.h5')

In [9]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("model.h5")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('/home/user_name/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x0000029C039355E0>.